In [19]:
from typing import Any

import faiss
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.prebuilt import ToolNode
from sentence_transformers import SentenceTransformer

In [2]:
load_dotenv()

True

In [3]:
data = {
    "product_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "product_name": [
        "Men's Classic Cotton T-Shirt",
        "Women's High-Waisted Jeans",
        "Waterproof Hiking Boots",
        "All-Weather Hiking Backpack",
        "Merino Wool Hiking Socks",
        "男士经典棉T恤",
        "女士高腰牛仔裤",
        "防水登山鞋",
        "智能手表",
        "无线蓝牙耳机",
    ],
    "description": [
        "A comfortable and durable t-shirt for everyday wear.",
        "Stylish and flattering jeans for any occasion.",
        "Durable, waterproof boots perfect for any trail or mountain.",
        "A spacious and comfortable backpack to carry all your gear.",
        "Warm and breathable socks designed for long hikes.",
        "舒适耐穿的日常T恤。",
        "适合任何场合的时尚修身牛仔裤。",
        "耐用、防水的靴子，适合任何小径或山脉。",
        "一款多功能智能手表，可追踪您的健康和通知。",
        "提供高品质音效和长续航的无线耳机。",
    ],
    "category": [
        "Apparel",
        "Apparel",
        "Footwear",
        "Gear",
        "Apparel",
        "服装",
        "服装",
        "鞋类",
        "电子产品",
        "电子产品",
    ],
    "price": [
        25.00,
        75.00,
        120.00,
        90.00,
        15.00,
        150.00,
        450.00,
        700.00,
        1500.00,
        800.00,
    ],
}
product_catalog = pd.DataFrame(data)

In [7]:
class Recommender:
    def __init__(
        self,
        product_catalog: pd.DataFrame,
        model_name: str = "paraphrase-multilingual-mpnet-base-v2",
    ) -> None:
        self.product_catalog = product_catalog
        self.model = SentenceTransformer(model_name)
        self._create_vector_store()

    def _create_vector_store(self) -> None:
        self.product_catalog["embedding_text"] = (
            self.product_catalog["product_name"]
            + " | "
            + self.product_catalog["description"]
        )
        embeddings = self.model.encode(
            self.product_catalog["embedding_text"].tolist(),
            show_progress_bar=False,
        )
        self.vector_store = faiss.IndexFlatL2(embeddings.shape[1])
        self.vector_store.add(np.array(embeddings, dtype=np.float32))

    def recommend_products(self, query: str, k: int = 3) -> list[dict[str, Any]]:
        query_embedding = self.model.encode([query])
        _, indices = self.vector_store.search(
            np.array(query_embedding, dtype=np.float32),
            k,
        )
        results = [self.product_catalog.iloc[i].to_dict() for i in indices[0]]
        for r in results:
            del r["embedding_text"]  # clean up the dict for LLM processing
        return results

In [8]:
recommender = Recommender(product_catalog)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
@tool
def recommend_products_tool(query: str) -> list[dict[str, Any]]:
    """Use this tool to find products based on what the user is looking for."""
    print(f"\nTOOL CALLED: recommend_products_tool with query='{query}'")
    return recommender.recommend_products(query)

In [12]:
tools = [recommend_products_tool]
tool_node = ToolNode(tools)

In [13]:
class AgentState(dict):
    @property
    def messages(self) -> list[BaseMessage]:
        return self.get("messages", [])

In [15]:
llm = ChatAnthropic(model="claude-3-7-sonnet-latest", temperature=0.7)

In [33]:
def call_model(state: AgentState):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """"You are a friendly and helpful e-commerce assistant.
                Your goal is to help users find products.
                You MUST use the 'recommend_products_tool' to find products for a user.
                Do not make up products.
                If the user asks a question in Chinese, you MUST respond in Chinese.
                """,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ],
    )
    chain = prompt | llm.bind_tools(tools)
    print(state)
    response = chain.invoke({"messages": state["messages"]})
    return {"messages": [response]}

In [34]:
workflow = StateGraph(AgentState)
workflow.add_node("call_model", call_model)
workflow.add_node("action", tool_node)
workflow.set_entry_point("call_model")
workflow.add_conditional_edges(
    "call_model",
    lambda x: "tool_calls" in x["messages"][-1].additional_kwargs,
    {True: "action", False: END},
)
workflow.add_edge("action", "call_model")
app = workflow.compile()

In [35]:
query = "Hi there! I'm planning a hiking trip and need some gear."
inputs = {"messages": [HumanMessage(content=query)]}

In [36]:
app.invoke(inputs)

{}


KeyError: 'messages'

In [30]:
app.stream(inputs)

<generator object Pregel.stream at 0x7f7286a5ab60>

In [31]:
for event in app.stream(inputs):
    for key, value in event.items():
        print(f"--- Node: {key} ---")
        print(f"{value}\n")

KeyError: 'messages'

In [ ]:
def run_conversation_and_print_trace(input_text: str):
    inputs = {"messages": [HumanMessage(content=input_text)]}
    print("\n--- LangGraph Trace ---")
    for event in app.stream(inputs):
        for key, value in event.items():
            print(f"--- Node: {key} ---")
            print(f"{value}\n")
    print("--- End of Conversation ---")

In [ ]:
run_conversation_and_print_trace()
run_conversation_and_print_trace("你好，我正在寻找一款新的智能手表。")



--- Starting Conversation: User says 'Hi there! I'm planning a hiking trip and need some gear.' ---

--- LangGraph Trace ---


KeyError: 'messages'